In [1]:
import os
# ggf muss Pfad angepasst werden
os.chdir("{}/..".format(os.getcwd()))
os.getcwd()
import pandas as pd
import numpy as np
from sklearn import model_selection, linear_model, metrics, svm
from scipy.stats import uniform, randint
from sklearn.model_selection import cross_validate
import xgboost as xgb
from sklearn.preprocessing import MinMaxScaler

In [2]:
%run notebooks/utils.ipynb

## Load Data

In [3]:
df_t_train = pd.read_csv("data/train_new.csv", sep="|")
df_t_val = pd.read_csv("data/val_new.csv", sep="|")
df_test = pd.read_csv("data/test.csv", sep="|")

df_t_train = add_new_features(df_t_train)
df_t_val = add_new_features(df_t_val)
df_test = add_new_features(df_test)

## Create Input x and labels y

In [4]:
my_scaler = MinMaxScaler()

train_y = df_t_train.fraud
train_x = df_t_train.drop(columns=['fraud'])

val_y = df_t_val.fraud
val_x = df_t_val.drop(columns=['fraud'])

df_test_x = df_test

## Scale for SVM


In [5]:
my_scaler = MinMaxScaler()
my_scaler.fit(train_x.append(df_test, sort=False))

train_x_scaled = scale_dataframe(train_x,my_scaler)
tmp_train = scale_dataframe(train_x,my_scaler)
tmp_train['fraud'] = train_y

val_x_scaled = scale_dataframe(val_x,my_scaler)
tmp_val = scale_dataframe(val_x,my_scaler)
tmp_val['fraud'] = val_y
df_test_x_scaled = scale_dataframe(df_test,my_scaler)

/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/data.py:334: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)


## Define Classifiers

In [6]:
linear_svc = svm.SVC(C=56.98164719395536, cache_size=8000, class_weight=None, coef0=0.0, \
                     decision_function_shape='ovr', degree=3, gamma='auto_deprecated', \
                     kernel='linear', max_iter=-1, probability=True, random_state=None, shrinking=True, tol=0.002848504943774676, verbose=0)

xgb = xgb.XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, disable_default_eval_metric=1,
       eta=0.1769182150877735, eval_metric='aucpr',
       gamma=1.8285912697052542,
       learning_rate=0.1, max_bin=254, max_delta_step=7.2556696256684035,
       max_depth=3, min_child_weight=1.0317712458399741, missing=None,
       n_estimators=445, n_jobs=-1, nthread=None,
       objective='binary:logistic', random_state=0, reg_alpha=0,
       reg_lambda=1, scale_pos_weight=1, seed=None, silent=True,
       subsample=1, tree_method='gpu_hist', verbosity=2)

In [17]:
res = test_classification(linear_svc, tmp_train, tmp_val)

Results Fix Split: 
DMC Score: 70  ---  Normalized DMC Score: 0.18617021276595744, 

Results Cross Validation: 
DMC Score: 55.0  ---  Normalized DMC Score: 0.1462836879432624 


In [12]:
res2 = test_classification(xgb, df_t_train, df_t_val)

Results Fix Split: 
DMC Score: 15  ---  Normalized DMC Score: 0.0398936170212766, 

Results Cross Validation: 
DMC Score: 47.0  ---  Normalized DMC Score: 0.125 


In [18]:
df1 = res['dataframe']
df2 = res2['dataframe']

In [19]:
df1[(df1.prediction != df1.fraud)]

,trustLevel,totalScanTimeInSeconds,grandTotal,lineItemVoids,scansWithoutRegistration,quantityModifications,scannedLineItemsPerSecond,valuePerSecond,lineItemVoidsPerPosition,totalScannedItems,fraud,prediction,probablity
97,0.4,0.952459,0.784278,0.818182,0.8,0.8,0.000555,0.000451,0.027273,1.000000,0,1,0.563
312,0.2,0.797814,0.398240,0.545455,0.9,0.2,0.000552,0.000273,0.021818,0.827586,1,0,0.622
353,0.0,0.503825,0.726273,0.454545,0.9,0.6,0.000740,0.000789,0.021645,0.689655,1,0,0.581


In [16]:
df2[(df2.prediction != df2.fraud)]

,trustLevel,totalScanTimeInSeconds,grandTotal,lineItemVoids,scansWithoutRegistration,quantityModifications,scannedLineItemsPerSecond,valuePerSecond,lineItemVoidsPerPosition,fraud,totalScannedItems,prediction,probablity
87,1,1486,57.13,4,1,1,0.015478,0.038445,0.173913,0,23.0,1,0.539
97,3,1744,78.42,9,8,4,0.017202,0.044966,0.300000,0,30.0,1,0.563
133,2,1819,60.15,0,7,3,0.016493,0.033068,0.000000,1,30.0,0,0.713
176,1,1625,89.93,11,3,0,0.011692,0.055342,0.578947,1,19.0,0,0.710
200,1,1538,60.68,8,6,5,0.012354,0.039454,0.421053,1,19.0,0,0.724
208,2,1641,51.34,10,6,2,0.015235,0.031286,0.400000,1,25.0,0,0.929
353,1,923,72.62,5,9,3,0.022752,0.078678,0.238095,1,21.0,0,0.576
